# Deploy

Deploying a model, following https://learn.microsoft.com/en-us/azure/machine-learning/tutorial-deploy-model?view=azureml-api-2

**NOTE**: DON'T RUN THIS YET! 

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="1b1ae7cf-df24-428b-8bb9-e4dd07869ac9",
    resource_group_name="SummerProjects2023",
    workspace_name="Nanostics_ML_Workspace",
)

In [ ]:
registered_model_name = "wisconsin-bca"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

print(latest_model_version)

## Creating an Endpoint

This is where we can communicate with our model

In [ ]:
import uuid

# Create a unique name for the endpoint
online_endpoint_name = "wisconsin-bca-endpoint-" + str(uuid.uuid4())[:8]

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# define an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="an endpoint for our Wisconsin BCa model",
    auth_mode="key",
    tags={
        "training_dataset": "wisconsin-bca",
    },
)

## Create Endpoint

In [ ]:
# create the online endpoint
# expect the endpoint to take approximately 2 minutes.

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

## Deploying!

Deploying our model (wisconsin-bca) to the endpoint above

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment

# Choose the latest version of our registered model for deployment
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# define an online deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

In [ ]:
# create the online deployment
blue_deployment = ml_client.online_deployments.begin_create_or_update(
    blue_deployment
).result()

# blue deployment takes 100% traffic
# expect the deployment to take approximately 8 to 10 minutes.
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
# return an object that contains metadata for the endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# print a selection of the endpoint's metadata
print(
    f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}"
)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

### Testing the endpoint

In [ ]:
# test the blue deployment with the sample data
ml_client.online_endpoints.invoke(
    endpoint_name='wisconsin-bca-endpoint-c1f57e99',
    deployment_name="blue",
    request_file="./deploy/sample-request.json"
)
